In [1]:
import ffmpeg

def convert_video_for_youtube(input_path, output_path, resolution="1920x1080", video_bitrate="10M", audio_bitrate="320k"):
    """
    Convert a video to YouTube recommended settings using ffmpeg-python.

    :param input_path: Path to the input video file.
    :param output_path: Path to save the converted video file.
    :param resolution: Target resolution (e.g., "1920x1080").
    :param video_bitrate: Target video bitrate (e.g., "8M").
    :param audio_bitrate: Target audio bitrate (e.g., "192k").
    """
    try:
        # Split resolution into width and height
        width, height = map(int, resolution.split('x'))
        
        # Create FFmpeg input stream
        input_stream = ffmpeg.input(input_path)
        
        # Create FFmpeg output stream with specified settings
        output_stream = ffmpeg.output(
            input_stream,
            output_path,
            vcodec='libx264',
            acodec='aac',
            video_bitrate=video_bitrate,
            audio_bitrate=audio_bitrate,
            vf=f'scale={width}:{height}',
            preset='slow',
            crf=18,
            movflags='+faststart'
        )
        
        # Execute the FFmpeg command
        ffmpeg.run(output_stream, overwrite_output=True)
        print(f"Video has been successfully converted and saved to {output_path}")
        
    except ffmpeg.Error as e:
        print(f"An error occurred: {e.stderr.decode()}")

# Example usage
input_video_path = "video.mp4"
output_video_path = "video2.mp4"

convert_video_for_youtube(input_video_path, output_video_path)


Video has been successfully converted and saved to video2.mp4


In [3]:
import os
from moviepy.editor import VideoFileClip
import imagehash
from PIL import Image
import numpy as np

def compare_videos_properties(video1_path, video2_path):
    """
    Compare basic properties of two videos.
    """
    video1 = VideoFileClip(video1_path)
    video2 = VideoFileClip(video2_path)

    properties = [
        ("duration", video1.duration, video2.duration),
        ("fps", video1.fps, video2.fps),
        ("resolution", video1.size, video2.size),
        ("filesize", os.path.getsize(video1_path), os.path.getsize(video2_path))
    ]

    for prop, val1, val2 in properties:
        print(f"{prop.capitalize()}: {val1} vs {val2}")
        if val1 != val2:
            print(f"Difference detected in {prop}!")

def frame_to_imagehash(frame, hash_size=8):
    """
    Convert a video frame to an image hash.
    """
    image = Image.fromarray(frame)
    return imagehash.phash(image, hash_size=hash_size)

def compare_video_frames(video1_path, video2_path, frame_interval=1, hash_size=8):
    """
    Compare frames of two videos to detect visual differences.
    """
    video1 = VideoFileClip(video1_path)
    video2 = VideoFileClip(video2_path)

    duration = min(video1.duration, video2.duration)

    for t in np.arange(0, duration, frame_interval):
        frame1 = video1.get_frame(t)
        frame2 = video2.get_frame(t)

        hash1 = frame_to_imagehash(frame1, hash_size=hash_size)
        hash2 = frame_to_imagehash(frame2, hash_size=hash_size)

        if hash1 != hash2:
            print(f"Frame difference detected at time {t:.2f}s!")
            print(f"Hash1: {hash1}, Hash2: {hash2}")
            return False

    return True

# Example usage
uploaded_video_path = "video.mp4"
downloaded_video_path = "video7.mp4"

# Compare basic properties
compare_videos_properties(uploaded_video_path, downloaded_video_path)

# Compare video frames
frames_match = compare_video_frames(uploaded_video_path, downloaded_video_path)
if frames_match:
    print("Videos are visually identical.")
else:
    print("Videos are not visually identical.")


Duration: 15.77 vs 15.77
Fps: 25.0 vs 25.0
Resolution: [1920, 1080] vs [1280, 720]
Difference detected in resolution!
Filesize: 9218257 vs 3692833
Difference detected in filesize!
Frame difference detected at time 1.00s!
Hash1: d59533c3884f585b, Hash2: d59533c3884f685b
Videos are not visually identical.
